In [10]:
library('RODBC')
library('quantmod')
library('PerformanceAnalytics')
source("/usr/common/config.R")
lcon <- odbcDriverConnect(sprintf("Driver={SQL Server};Server=%s;Database=%s;Uid=%s;Pwd=%s;", ldbserver, ldbname, ldbuser, ldbpassword), case = "nochange", believeNRows = TRUE)

Select 10 equity funds with the highest AUM on 2011/3/1 (since the M100 ETF was listed only on 2011/2.) 
Ignore sectoral and ELSS funds.

In [11]:
startDate<-as.Date('2011-03-01')

schemes<-data.frame(SCHEME_CODE=0, SCHEME_NAME="", stringsAsFactors = F)
schemes<-rbind(schemes, c(102000, 'HDFC Top 200'))
schemes<-rbind(schemes, c(101762, 'HDFC Equity'))
schemes<-rbind(schemes, c(100377, 'Reliance Growth'))
schemes<-rbind(schemes, c(103085, 'Reliance Regular Savings - EQUITY'))
schemes<-rbind(schemes, c(100471, 'Franklin India Bluechip'))
schemes<-rbind(schemes, c(103174, 'Birla Sun Life Frontline Equity'))
schemes<-rbind(schemes, c(101161, 'Reliance Equity Opportunities'))
schemes<-rbind(schemes, c(100380, 'Reliance Vision'))
schemes<-rbind(schemes, c(101635, 'DSP BlackRock Top 100 Equity'))
schemes<-rbind(schemes, c(102434, 'DSP BlackRock India T.I.G.E.R.'))
schemes<-schemes[-1,]

mfRets<-xts()
for(i in 1:length(schemes[,1])){
    scode<-schemes[i,1]

    nav<-sqlQuery(lcon, sprintf("select NAV, AS_OF from MF_NAV_HISTORY
                                    where SCHEME_CODE = %s
                                    and AS_OF >= '%s'", scode, startDate))
    
    navXts<-xts(nav$NAV, as.Date(nav$AS_OF))
    mfRets<-merge(mfRets, dailyReturn(navXts))
}
names(mfRets)<-c(schemes[,1])

Warning message in attributes(.Data) <- c(attributes(.Data), attrib):
“closing unused RODBC handle 3”

NIFTYBEES, JUNIORBEES and M100 happen to be fairly liquid ETFs that span the large & midcap spectrum.

In [12]:
etfs<-c('NIFTYBEES', 'JUNIORBEES', 'M100')

etfRets<-xts()
for(etf in etfs){
    nav<-sqlQuery(lcon, sprintf("select TIME_STAMP, PX_CLOSE from PX_HISTORY
                                    WHERE SYMBOL='%s'
                                    AND TIME_STAMP >= '%s'", etf, startDate))
    
    navXts<-xts(nav$PX_CLOSE, as.Date(nav$TIME_STAMP))
    etfRets<-merge(etfRets, dailyReturn(navXts))
}

names(etfRets)<-etfs

Regress the 200 days of returns over the first 500 days of returns

In [37]:
returnDf<-data.frame(SCHEME_CODE="", FACTORS="", MF_RET_CUM=0.0, ETF_RET_CUM=0.0, MF_RET_ANN=0.0, ETF_RET_ANN=0.0, DIFF=0.0, stringsAsFactors = F)
for(i in 1:length(schemes[,1])){
    scode<-schemes[i,1]
    lmFormula<-formula(sprintf("X%s ~ %s + 0", scode, paste(etfs, collapse="+")))
    
    lmSubset<-na.omit(merge(mfRets[1:500, scode], etfRets[1:500,]))
    
    mf1<-rollapply(lmSubset[,1], 200, Return.cumulative)
    etf1<-rollapply(lmSubset[,2], 200, Return.cumulative)
    etf2<-rollapply(lmSubset[,3], 200, Return.cumulative)
    etf3<-rollapply(lmSubset[,4], 200, Return.cumulative)
    
    lmSubsetRoll<-na.omit(merge(mf1, etf1, etf2, etf3))
    
    names(lmSubsetRoll)<-c(scode, etfs)
    
    lmret<-lm(lmFormula, data=data.frame(lmSubsetRoll))$coefficients
    
    #handle -ves
    aF<-max(0, lmret[1])
    bF<-max(0, lmret[2])
    cF<-max(0, lmret[3])
    
    aF1<-aF/(aF+bF+cF)
    bF1<-bF/(aF+bF+cF)
    cF1<-cF/(aF+bF+cF)
        
    allXts<-na.omit(merge(mfRets[-(1:500), scode], aF1*etfRets[-(1:500), 1] + bF1*etfRets[-(1:500), 2] + cF1*etfRets[-(1:500), 3]))

    cumMf<-Return.cumulative(allXts[,1])
    cumEtfs<-Return.cumulative(allXts[,2])

    annMf<-Return.annualized(allXts[,1])
    annEtfs<-Return.annualized(allXts[,2])

    returnDf<-rbind(returnDf, c(scode, sprintf("%.2f=>%.2f, %.2f=>%.2f, %.2f=>%.2f", lmret[1], aF1, lmret[2], bF1, lmret[3], cF1), round(100*cumMf, 2), round(100*cumEtfs, 2), round(100*annMf, 2), round(100*annEtfs, 2), round(100*(annMf-annEtfs)/annMf, 4)))
}
returnDf<-returnDf[-1,]
print(returnDf)

   SCHEME_CODE                             FACTORS MF_RET_CUM ETF_RET_CUM
2       102000 0.74=>0.69, 0.33=>0.31, -0.12=>0.00      91.49       79.81
3       101762  0.59=>0.57, 0.26=>0.25, 0.18=>0.18     104.28       91.18
4       100377 0.10=>0.12, 0.80=>0.88, -0.08=>0.00     122.08      109.76
5       103085 0.09=>0.07, 1.17=>0.93, -0.17=>0.00        109      112.02
6       100471 0.80=>0.66, 0.40=>0.34, -0.47=>0.00       82.3       81.34
7       103174 0.76=>0.45, 0.92=>0.55, -0.69=>0.00     106.03       92.38
8       101161 0.11=>0.05, 2.11=>0.95, -1.55=>0.00      98.59      113.07
9       100380  0.22=>0.26, 0.16=>0.19, 0.47=>0.55     109.77      118.53
10      101635 0.54=>0.50, 0.54=>0.50, -0.44=>0.00      73.93       89.94
11      102434 0.47=>0.41, -0.02=>0.00, 0.68=>0.59     111.02      109.58
   MF_RET_ANN ETF_RET_ANN     DIFF
2       17.19       15.41  10.3895
3       19.06       17.14  10.0342
4       21.51       19.83   7.8161
5       19.72       20.14  -2.1348
6       15.

4 out of 10 funds underperformed their ETF baskets and 2 out of 10 funds could be replaced by an ETF basket without compromising too much on returns.